In [8]:
import sys
sys.path.append('../../../')
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from time import time
from tqdm import tqdm
is_cuda = torch.cuda.is_available()
device = 'cuda' if is_cuda else 'cpu'
if not is_cuda:
    print("Warning: CUDA not available; falling back to CPU but this is likely to be very slow.")
    
torch.set_printoptions(precision=3, sci_mode=False)
np.set_printoptions(suppress=True)

In [2]:
data = torch.load('../data.pt')

for dataset in data:
    data[dataset] = data[dataset].float()

In [4]:
from lib.Signature import Signature, Basepoint, sig_lreg, LeadLag, Cumsum2
from lib.Baseline import ConditionalGenerator
from lib.Utilities import get_n_params

In [5]:
hp = {'nsamples_fs': 100, 'batch_size': 64}

In [6]:
x_length = np.arange(2, 400, 20).tolist()
x_length_time = torch.zeros([len(x_length), 25])
x_length_memory = torch.zeros([len(x_length), 25])

In [17]:
for i in range(len(x_length)):
    
    x_length_ = x_length[i]
    data_ = {}
    for dataset in data:
        if dataset in ['X_train', 'X_val', 'X_test']:
            data_[dataset] = torch.clone(data[dataset][:, -x_length_:])
        else:
            data_[dataset] = torch.clone(data[dataset])
    
    print(data_['X_train'].shape)
    
    sig_X = Signature(depth=5, augmentations = [Basepoint, Cumsum2], 
                  data_size=data_['X_train'].shape[2],
                  interval=[0, data_['X_train'].shape[1]+1], 
                  q=1, 
                  t_norm = data_['X_train'][:, :, 0].max()).to(device)

    sig_Y = Signature(depth=4, augmentations = [Cumsum2], 
                  data_size=data_['Y_train'].shape[2],
                  interval=[0, data_['Y_train'].shape[1]+1], 
                  q=1, 
                  t_norm = data_['Y_train'][:, :, 0].max()).to(device)
    
    signatures_X, signatures_Y, signatures_Y_pred, sig_Y = sig_lreg(sig_X, sig_Y, data_, 528, alpha=0.1, normalize_sig = True)
    
    
    
    G = ConditionalGenerator(1, 1, 32, 5, 5).to('cuda')
    
    G_optimizer = torch.optim.Adam(G.parameters(), lr=1e-3)

    q = data_['Y_train'].shape[1]-1

    train_dataset = TensorDataset(data_['X_train'][:, :, 1:], signatures_Y_pred['train'])
    train_dataloader = DataLoader(train_dataset, batch_size = hp['batch_size'], shuffle=True)
    infinite_dataloader = (elem for it in iter(lambda: train_dataloader, None) for elem in it)

    for step in range(25):
        start_time = time()
        
        G_optimizer.zero_grad()
        X_batch, sigY_pred_batch = next(infinite_dataloader) 
        X_batch_mc = X_batch.repeat(hp['nsamples_fs'], 1, 1).requires_grad_().to(device)
        del X_batch
        Y_batch_pred = G(X_batch_mc, q)
        del X_batch_mc
        t = torch.arange(0, Y_batch_pred.shape[1]).repeat(Y_batch_pred.shape[0]).view(Y_batch_pred.shape[0], Y_batch_pred.shape[1], 1).to(device)
        Y_batch_pred_t = torch.cat([t, Y_batch_pred], dim=2)
        del t
        
        sigY_batch = sig_Y(Y_batch_pred_t).view(hp['nsamples_fs'], sigY_pred_batch.shape[0], sigY_pred_batch.shape[1])
        del Y_batch_pred_t
        sigY_batch_mean = torch.mean(sigY_batch, dim=0)
        del sigY_batch
        sigY_pred_batch = sigY_pred_batch.to(device)
        loss = torch.sum(torch.norm(sigY_batch_mean-sigY_pred_batch, p=2, dim=1))
        loss.backward()
        del sigY_pred_batch, sigY_batch_mean

        G_optimizer.step()
        
        end_time = time()
        x_length_time[i, step] = end_time-start_time
        x_length_memory[i, step] = torch.cuda.max_memory_allocated(device='cuda')*1e-6
        torch.cuda.reset_max_memory_allocated(device='cuda')
    

In [18]:
torch.save(torch.mean(x_length_time, dim=1)[:18], 'x_length_time.pt')

In [19]:
torch.save(torch.mean(x_length_memory, dim=1)[:18], 'x_length_memory.pt')